In [1]:
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

#authenticate
credential = DefaultAzureCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id="c6fb9fd9-644b-44c5-8e1f-2ea146326c95",
    resource_group_name="Alexander.Dunnett-rg",
    workspace_name="demo-alexplore"
)

In [14]:
data_asset = ml_client.data.get(name="credit_card",version="2023.07.06.22.03.34_cleaned")
df = pd.read_parquet(data_asset.path)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('default',axis=1), df['default'], test_size=0.25, random_state=42)

In [26]:
#using MLflow to track development: w/ autologging
mlflow.set_experiment("Develop on cloud tutorial")
mlflow.sklearn.autolog()

2023/07/06 22:48:09 INFO mlflow.tracking.fluent: Experiment with name 'Develop on cloud tutorial' does not exist. Creating a new experiment.
2023/07/06 22:48:09 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


In [28]:
mlflow.start_run()
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
classification_report(y_test,y_pred)
mlflow.end_run()

2023/07/06 22:50:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/models/signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/06 22:50:05 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/anaconda/envs/azureml_py310_

In [29]:
#Try random forest
from sklearn.ensemble import RandomForestClassifier

mlflow.start_run()
rnd = RandomForestClassifier(n_estimators=100,max_depth=10,random_state=42)
rnd.fit(X_train,y_train)
y_pred = rnd.predict(X_test)
classification_report(y_test,y_pred)
mlflow.end_run()

2023/07/06 22:53:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/models/signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


In [30]:
classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n           0       0.84      0.95      0.89      5873\n           1       0.67      0.36      0.47      1627\n\n    accuracy                           0.82      7500\n   macro avg       0.76      0.65      0.68      7500\nweighted avg       0.81      0.82      0.80      7500\n'

Demo update for git push testing